In [5]:
import os
print(f"Current working folder: {os.getcwd()}")

os.chdir('C:/Users/UHY6HC/Downloads/Thesis/RAG_LLM_DA')
print(f"Current working folder: {os.getcwd()}")

from stages.stage_1_learn_rules_from_data.data_loader import DataLoader
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage
from openai_llm.llm_init import LLM_Model
from utils import load_json_data, save_json_data, load_vectorstore_db, lookup_vector_db, calculate_similarity

Current working folder: c:\Users\UHY6HC\Downloads\Thesis\RAG_LLM_DA\notebooks
Current working folder: C:\Users\UHY6HC\Downloads\Thesis\RAG_LLM_DA


In [6]:
llm_instance = LLM_Model()

Loading data from config/llm_config.json


In [7]:
data_dict = load_vectorstore_db(llm_instance, 'icews14')
for collection in data_dict:
    print(f"{collection}: {len(data_dict[collection]['vector_db'].get()['documents'])} documents")

Loading data from config/data_embedding.json
facts: 181460 documents
rules: 0 documents


In [8]:


# filter = {"$and": [
# 	{"object": {"$in": ["Malaysia", 'China']}},
# 	{"subject": {"$in": ["Malaysia", 'China']}},
#     # {"relation": "Express_intent_to_cooperate"}
# ]
# }

# filter = {"$or":[{"object": "Malaysia"}, {"subject": "Malaysia"}]}
filter = {"subject_id": 118}
docs = lookup_vector_db("", filter, data_dict['facts']['vector_db'], llm_instance, top_k=100)
docs = sorted(docs, key=lambda x: x.metadata['timestamp_id'], reverse=True)
docs

[Document(metadata={'object': 'Pope_Francis', 'object_id': 75, 'relation': 'inv_Host_a_visit', 'relation_id': 235, 'subject': 'Cristina_Fernández_de_Kirchner', 'subject_id': 118, 'timestamp': '2014-12-29', 'timestamp_id': 363}, page_content='"Cristina_Fernández_de_Kirchner was hosted a visit by to/with Pope_Francis on 2014-12-29"'),
 Document(metadata={'object': 'Pope_Francis', 'object_id': 75, 'relation': 'Make_a_visit', 'relation_id': 4, 'subject': 'Cristina_Fernández_de_Kirchner', 'subject_id': 118, 'timestamp': '2014-12-29', 'timestamp_id': 363}, page_content='"Cristina_Fernández_de_Kirchner made a visit to/with Pope_Francis on 2014-12-29"'),
 Document(metadata={'object': 'Government_(Argentina)', 'object_id': 190, 'relation': 'inv_Investigate', 'relation_id': 248, 'subject': 'Cristina_Fernández_de_Kirchner', 'subject_id': 118, 'timestamp': '2014-12-26', 'timestamp_id': 360}, page_content='"Cristina_Fernández_de_Kirchner was Investigated by to/with Government_(Argentina) on 2014-12

---

### Adding new documents

In [8]:
from langchain.vectorstores import Chroma
import os
from langchain_community.vectorstores.utils import filter_complex_metadata
from process_embedding.custom_embedding_function import CustomEmbeddingFunction

db_directory = "result/icews14/vector_database_build"
collection = "facts"

# Đường dẫn đến thư mục lưu trữ collection
collection_persist_directory = os.path.join(db_directory, collection)

# Khởi tạo Chroma với tham số CHÍNH XÁC như lúc tạo collection ban đầu
db = Chroma(persist_directory=collection_persist_directory, embedding_function=CustomEmbeddingFunction(llm_instance))

# Kiểm tra xem collection đã load đúng chưa
print("Số documents trước khi thêm:", db._collection.count())

# # Thêm documents mới (đảm bảo metadata khớp schema)
# new_docs = [...]
# db.add_documents(documents=new_docs)

# Kiểm tra lại
x = db.get()['ids']
numbers = sorted(int(num) for num in x)



Số documents trước khi thêm: 181440


In [9]:
missing_sequences = []
for i in range(1, len(numbers)):
    prev_num = numbers[i-1]
    current_num = numbers[i]
    gap = current_num - prev_num
    
    if gap > 1:
        start_missing = prev_num + 1
        end_missing = current_num - 1
        missing_count = end_missing - start_missing + 1
        missing_sequences.append((start_missing, end_missing, missing_count))
target_sequence = None
for start, end, count in missing_sequences:
    if count == 20:
        target_sequence = list(range(start, end + 1))
        break

if target_sequence:
    print("Dãy 20 số thiếu:", target_sequence)
else:
    print("Không tìm thấy dãy 20 số liên tiếp thiếu.")

Dãy 20 số thiếu: [89340, 89341, 89342, 89343, 89344, 89345, 89346, 89347, 89348, 89349, 89350, 89351, 89352, 89353, 89354, 89355, 89356, 89357, 89358, 89359]
